In [ ]:
import cv2
import numpy as np
import os
import pandas as pd

def global_contrast_std_luminance(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    luminance_std = np.std(gray)
    return luminance_std

def wcag_contrast_ratio(lum1, lum2):
    L1, L2 = max(lum1, lum2), min(lum1, lum2)
    return (L1 + 0.05) / (L2 + 0.05)

def contrast_textlike_vs_background(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contrast_values = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        # Filter kontur kecil (mirip karakter)
        if 5 < w < 100 and 5 < h < 100:
            roi = gray[y:y+h, x:x+w]
            text_luminance = np.mean(roi)

            bg_patch = gray[max(0, y-10):y, x:x+w]
            if bg_patch.size == 0:
                bg_patch = gray[min(y+h+1, gray.shape[0]-1):y+h+11, x:x+w]

            bg_luminance = np.mean(bg_patch) if bg_patch.size > 0 else 255
            contrast = wcag_contrast_ratio(text_luminance / 255.0, bg_luminance / 255.0)
            contrast_values.append(contrast)

    return np.mean(contrast_values) if contrast_values else 0.0

def bright_dark_ratio(image_path, threshold=127):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    total_pixels = gray.size
    bright_pixels = np.sum(gray > threshold)
    return bright_pixels / total_pixels

def process_contrast_features(folder_path, output_csv):
    data = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(folder_path, filename)
            name = os.path.splitext(filename)[0]
            try:
                image_number = int(name)
            except ValueError:
                continue

            std_contrast = global_contrast_std_luminance(path)
            contrast_text = contrast_textlike_vs_background(path)
            bright_ratio = bright_dark_ratio(path)

            data.append({
                'image_name': image_number,
                'luminance_std': std_contrast,
                'contrast_text_vs_background': contrast_text,
                'bright_dark_ratio': bright_ratio
            })

    df = pd.DataFrame(data)
    df = df.sort_values(by='image_name').reset_index(drop=True)
    df.to_csv(output_csv, index=False)
    print(f"✅ CSV berhasil disimpan: {output_csv}")

In [ ]:
folder_path = '../Datasets/datasets'
output_csv = 'contrast_features.csv'
process_contrast_features(folder_path, output_csv)

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.